In [1]:
from pathlib import Path
import numpy as np
import itertools
from collections import defaultdict

In [2]:
def read(prefix='data'):
    data = Path(f'{prefix}/17.txt').read_text().rstrip()
    return data

In [3]:
disp = lambda B, h=10 : print(*[''.join(r) for r in B[-h:]], sep='\n')

In [4]:
def board(count=2022):
    B = np.full((count*4+10,9), fill_value='.')
    B[:,0] = '|'
    B[:,-1] = '|'
    B[-1,:] = '-'
    B[-1,0] = '+'
    B[-1,-1] = '+'
    return B

In [5]:
def shapes(count=2022):
    S = {}
    S['-'] = np.array([['#','#','#','#']])
    S['+'] = np.array([['.','#','.'],['#', '#', '#'], ['.','#','.']])
    S['L'] = np.array([['.','.','#'],['.','.','#'],['#', '#', '#']])
    S['|'] = np.array([['#','#','#','#']]).T
    S['o'] = np.array([['#', '#'], ['#', '#']])
    return S

In [6]:
for s in shapes().values():
    print()
    disp(s)


####

.#.
###
.#.

..#
..#
###

#
#
#
#

##
##


In [7]:
disp(board(), h=10)

|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
|.......|
+-------+


In [8]:
def outofbounds(B, S, x, y):
    if x < 1:
        return True
    if y < 0:
        return True
    if x + S.shape[1] >= B.shape[1]:
        return True
    if y + S.shape[0] >= B.shape[0]:
        return True

    return False

In [9]:
def intersect(B, s, x, y):
    Bw = B[y:y+s.shape[0],x:x+s.shape[1]]
    return ((Bw == '#') & (s == '#')).any()

In [10]:
def place(B, s, x, y):
    Bw = B[y:y+s.shape[0],x:x+s.shape[1]]
    B[y:y+s.shape[0],x:x+s.shape[1]] = np.where((s == '#'), s, Bw)

In [11]:
def drop(B, s, M, M_idx, y_full):
    x = 3
    y = y_full - s.shape[0] - 3
    down = False
    while True:
        if down:
            xn, yn = x, y+1
        if not down:
            xn = x+1 if (M[M_idx] == '>') else x-1
            yn = y
            M_idx = (M_idx + 1) % len(M)
        oob = outofbounds(B,s, xn,yn)
        isct = intersect(B,s, xn, yn)
        if oob or isct:
            if down:
                break
        else:
            x, y = xn, yn

        down = not down

    place(B, s, x, y)
    y_full = min(y_full, y)
    return y_full, M_idx


In [12]:
def compute(M,count, M_idx=0):
    B = board(count)
    S = shapes()
    y_full = B.shape[0]-1 # floor
    seen = defaultdict(list)
    for i, (k,s) in zip(range(count), itertools.cycle(S.items())):
        y_full, M_idx = drop(B, s, M, M_idx, y_full)
        mins = tuple((B[:,1:-1] == '#').argmax(axis=0) - y_full)
        height = B.shape[0] - y_full - 1
        seen[mins, k, M_idx].append((height, i+1))
    return height, B, seen

In [13]:
count = 2022
M = read('test')
height, B, seen = compute(M,count)
print(height)
disp(B,h=20)

3068
|...#...|
|#..####|
|#...#..|
|#...#..|
|#...##.|
|##..##.|
|######.|
|.###...|
|..#....|
|.####..|
|....##.|
|....##.|
|....#..|
|..#.#..|
|..#.#..|
|#####..|
|..###..|
|...#...|
|..####.|
+-------+


In [14]:
count = 2022
M = read()
height, B, seen = compute(M,count)
height

3085

In [24]:
list(seen.values())[0]

[(1, 1)]

In [25]:
def find(seen, target):
    known = [h for c in seen.values() for (h,m) in c if m == target]
    if len(known) > 0:
        return known[0]

    reps = {k: v for (k,v) in seen.items() if len(v) > 1}
    cycles = {(n[0]-c[0],n[1]-c[1]) for rep in reps.items() for (c,n) in zip(rep[1], rep[1][1:])}
    assert len(cycles) == 1
    cyc_height, cyc_moves = list(cycles)[0]
    
    for c in reps.items():
        base_moves = c[1][0][1]
        base_height = c[1][0][0]

        if (target - base_moves) % cyc_moves  == 0:
            needed = int((target - base_moves) / cyc_moves)
            height = base_height + needed*cyc_height
            moves = base_moves + needed*cyc_moves
            assert moves == target
            return height

In [26]:
count = 2022
M = read('test')
_, _, seen = compute(M,count)
find(seen, target=1000000000000)

1514285714288

In [32]:
count = 5000
M = read()
_, _, seen = compute(M,count)
find(seen, target=1000000000000)

1535483870924